In [2]:
# Dependencies
import requests
import json
import prettytable
import pandas as pd

In [3]:
# Series IDs are for the categories representative of the metrics used by US Bureau of Labor
# JTU000000480000000JOL: Job openings in the state of Texas in all non-farm industries, with companies of all sizes,
# Values shown in thousands

In [4]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['JTU000000480000000JOL'],"startyear":"2018", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 215,
 'message': [],
 'Results': {'series': [{'seriesID': 'JTU000000480000000JOL',
    'data': [{'year': '2021',
      'period': 'M12',
      'periodName': 'December',
      'value': '818',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M11',
      'periodName': 'November',
      'value': '868',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M10',
      'periodName': 'October',
      'value': '965',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M09',
      'periodName': 'September',
      'value': '867',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M08',
      'periodName': 'August',
      'value': '970',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M07',
      'periodName': 'July',
      'value': '1000',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M06',
      'periodName': 'June',
      'value': '896',
      'footnotes': [

In [5]:
# There were 885,000 non-farm job openings in Texas in Dec 2022
json_data["Results"]["series"][0]["data"][0]["value"]

'818'

In [6]:
# Get the year
json_data["Results"]["series"][0]["data"][0]["year"]

'2021'

In [7]:
# Store the year, period, and periodName, and # of job openings in columns of the dataframe
year = []
period = []
periodName = []
value = []
num_series = len(json_data["Results"]["series"])

for i in range(num_series):
    series_data = json_data["Results"]["series"][i]["data"]
    for j in range(len(series_data)):
        year.append(series_data[j]["year"])
        period.append(series_data[j]["period"])
        periodName.append(series_data[j]["periodName"])
        value.append(series_data[j]["value"])



# Create a dataframe from these columns
job_openings = pd.DataFrame({"Year": year,
                            "Period": period,
                            "Month": periodName,
                            "# of job openings (thousands)": value})

# Since the last column contains string values, we convert them to integers
job_openings["# of job openings (thousands)"] = job_openings["# of job openings (thousands)"].astype(int)

# Show # of job openings in the actual thousand values
job_openings["# of job openings"] = job_openings["# of job openings (thousands)"]*1000

# Drop the duplicate column
job_openings = job_openings.drop(columns=["# of job openings (thousands)"])

# Print the dataframe
job_openings

,Year,Period,Month,# of job openings
0,2021,M12,December,818000
1,2021,M11,November,868000
2,2021,M10,October,965000
3,2021,M09,September,867000
4,2021,M08,August,970000
5,2021,M07,July,1000000
6,2021,M06,June,896000
7,2021,M05,May,814000
8,2021,M04,April,845000
9,2021,M03,March,718000


In [8]:
# Call the CPI from website and JSONify it
cpi_url = "https://data.texas.gov/resource/karz-jr5v.json"
cpi_data = requests.get(cpi_url).json()

cpi_data

[{'month': '1',
  'year': '2005',
  'nonfarm_employment_il_detail': '5829.60',
  'nonfarm_employment_fl_detail': '7650.20',
  'nonfarm_employment_ny_detail': '8490.60',
  'nonfarm_employment_tx_detail': '9642.40',
  'nonfarm_employment_ca_detail': '14881.80'},
 {'month': '2',
  'year': '2005',
  'nonfarm_employment_il_detail': '5836.20',
  'nonfarm_employment_fl_detail': '7670.70',
  'nonfarm_employment_ny_detail': '8487.90',
  'nonfarm_employment_tx_detail': '9653.20',
  'nonfarm_employment_ca_detail': '14908.30'},
 {'month': '3',
  'year': '2005',
  'nonfarm_employment_il_detail': '5833.80',
  'nonfarm_employment_fl_detail': '7668.70',
  'nonfarm_employment_ny_detail': '8483.60',
  'nonfarm_employment_tx_detail': '9670.80',
  'nonfarm_employment_ca_detail': '14929.30'},
 {'month': '4',
  'year': '2005',
  'nonfarm_employment_il_detail': '5857.50',
  'nonfarm_employment_fl_detail': '7716.40',
  'nonfarm_employment_ny_detail': '8518.60',
  'nonfarm_employment_tx_detail': '9715.10',
  '

In [36]:
# Create a dataframe for the CPI data of state of TX
year_cpi = []
month = []
cpi_tx = []

for i in range(1, len(cpi_data)): 
    cpi_data[i]["year"] = int(cpi_data[i]["year"])
    year = cpi_data[i]["year"]
    if (year >= 2018) and (year <= 2022):
        month.append(cpi_data[i]["month"])
        year_cpi.append(cpi_data[i]["year"])
        cpi_tx.append(cpi_data[i]["cpi_tx"])
        

cpi_tx_df = pd.DataFrame({"Year": year_cpi,
                         "Period": month,
                         "CPI of TX": cpi_tx})

cpi_tx_df

,Year,Period,CPI of TX
0,2018,1,225.954
1,2018,2,227.441
2,2018,3,227.746
3,2018,4,228.152
4,2018,5,230.043
5,2018,6,230.9
6,2018,7,230.489
7,2018,8,230.003
8,2018,9,230.566
9,2018,10,231.524


In [39]:
# Create a new Period column for merging purpose
cpi_tx_df['Period_New'] = cpi_tx_df['Period'].replace({'1': 'M01', '2': 'M02', '3': 'M03', '4': 'M04', '5': 'M05', '6': 'M06', '7': 'M07',
                            '8': 'M08', '9': 'M09', '10': 'M10', '11': 'M11', '12': 'M12'})
cpi_tx_df

,Year,Period,CPI of TX,Period_New
0,2018,1,225.954,M01
1,2018,2,227.441,M02
2,2018,3,227.746,M03
3,2018,4,228.152,M04
4,2018,5,230.043,M05
5,2018,6,230.9,M06
6,2018,7,230.489,M07
7,2018,8,230.003,M08
8,2018,9,230.566,M09
9,2018,10,231.524,M10
